In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from UnarySim.sw.bitstream.gen import RNG, BSGen


In [3]:
rng = RNG(4, 2, "Sobol").Out()
print(rng)

rng1 = RNG(4, 2, "Race").Out()
print(rng1)

tensor([ 8,  4, 12,  6, 14,  2, 10,  5, 13,  1,  9,  3, 11,  7, 15,  7])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])


In [4]:
bs1 = BSGen(torch.tensor([14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14]), rng1)

In [10]:
bs1.Gen(rng)

tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=torch.uint8)